In [ ]:
# staging.ipynb
# ----------------------
# 1. Подключение к PostgreSQL
# ----------------------
import pandas as pd
from torgstat.db import get_engine, DB_SCHEMA
import sys
from sqlalchemy import text

try:
    # Создаём движок SQLAlchemy
    engine = get_engine()
    
    # Проверяем реальное подключение с использованием text()
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1 as test_connection"))
        test_result = result.scalar()
        
    if test_result == 1:
        print("✓ Подключение к PostgreSQL успешно установлено и проверено")
    else:
        print("✗ Ошибка: Не удалось проверить подключение")
        sys.exit(1)
        
except Exception as e:
    print(f"✗ Ошибка подключения к PostgreSQL: {e}")
    sys.exit(1)

# Функция для чтения таблицы из схемы analytics с детальным логированием
def read_table(table_name: str):
    try:
        query = f'SELECT * FROM {DB_SCHEMA}.{table_name};'
        df = pd.read_sql(query, engine)
        
        # Детальная информация о сырых данных
        print(f"\n📊 {table_name.upper()} - СЫРЫЕ ДАННЫХ:")
        print(f"   📏 Строк: {len(df):,}")
        print(f"   🧩 Столбцов: {len(df.columns)}")
        
        # Анализ пропусков
        null_counts = df.isnull().sum()
        if null_counts.sum() > 0:
            print(f"   ⚠️  Пропуски:")
            for col, count in null_counts.items():
                if count > 0:
                    print(f"      - {col}: {count} пропусков ({count/len(df)*100:.1f}%)")
        
        # Анализ типов данных
        print(f"   🔧 Типы данных:")
        for col, dtype in df.dtypes.items():
            print(f"      - {col}: {dtype}")
            
        # Особые проверки для ключевых полей
        if 'amount' in df.columns:
            amount_stats = df['amount'].apply(lambda x: type(x).__name__).value_counts()
            print(f"   💰 Типы значений в amount: {amount_stats.to_dict()}")
            
        if 'status' in df.columns:
            print(f"   🏷️  Уникальные статусы: {df['status'].unique()}")
            
        print(f"✓ Таблица {table_name} успешно загружена")
        return df
        
    except Exception as e:
        print(f"✗ Ошибка загрузки таблицы {table_name}: {e}")
        raise

In [ ]:
# ----------------------
# 2. Читаем сырые таблицы с обработкой ошибок
# ----------------------

tables_to_load = ["users", "sessions", "subscriptions", "invoices", "plans", "events"]
loaded_tables = {}
failed_tables = []

for table_name in tables_to_load:
    try:
        df = read_table(table_name)
        loaded_tables[table_name] = df
    except Exception as e:
        print(f"✗ Критическая ошибка загрузки {table_name}: {e}")
        failed_tables.append(table_name)

# Проверяем результаты
if failed_tables:
    print(f"\n❌ НЕ ЗАГРУЖЕНЫ таблицы: {failed_tables}")
    print("Работа прервана из-за критических ошибок")
    sys.exit(1)
else:
    print("\n" + "="*50)
    print("✅ ВСЕ таблицы успешно загружены и проанализированы")
    print("="*50)

# Просмотр первых строк (ИСПОЛЬЗУЕМ СЛОВАРЬ loaded_tables)
for table_name, df in loaded_tables.items():
    print(f"\n📋 {table_name.upper()} (первые 5 строк):")
    display(df.head())

print("\n🔍 ДЕТАЛЬНЫЙ АНАЛИЗ ПРОБЛЕМ:")
print("═" * 50)

# Анализ amount в invoices (используем словарь)
if 'amount' in loaded_tables["invoices"].columns:
    print("💰 INVOICES.amount:")
    print(f"   Тип данных: {loaded_tables['invoices']['amount'].dtype}")
    print(f"   Уникальные типы значений: {loaded_tables['invoices']['amount'].apply(type).value_counts().to_dict()}")
    print(f"   Отрицательные значения: {(pd.to_numeric(loaded_tables['invoices']['amount'], errors='coerce') < 0).sum()}")
    
# Анализ дат (используем словарь)
date_columns = ['signup_date', 'session_date', 'start_date', 'period_start', 'period_end', 'invoice_date', 'event_date']
for col in date_columns:
    for table_name, df in loaded_tables.items():
        if col in df.columns:
            print(f"📅 {table_name}.{col}:")
            print(f"   Тип: {df[col].dtype}")
            print(f"   Примеры: {df[col].head(3).tolist()}")

In [ ]:
# ----------------------
# 3. Создаём staging-таблицы
# ----------------------
from sqlalchemy import types
import numpy as np

# STG_USERS: очистка и нормализация
df_stg_users = loaded_tables["users"].copy()
df_stg_users['signup_date'] = pd.to_datetime(df_stg_users['signup_date'], errors='coerce')
df_stg_users = df_stg_users.drop_duplicates(subset='user_id')
print(f"STG_USERS: {len(df_stg_users)} строк после очистки")
# ДЕТАЛЬНОЕ ЛОГИРОВАНИЕ
print(f"   📅 signup_date преобразован в: {df_stg_users['signup_date'].dtype}")
print(f"   🗑️  Удалено дубликатов: {len(loaded_tables["users"]) - len(df_stg_users)}")
print(f"   ⚠️  Пропуски region: {df_stg_users['region'].isnull().sum()}")

# STG_SESSIONS: проверяем дубликаты, типы данных
df_stg_sessions = loaded_tables["sessions"].copy()
df_stg_sessions['session_date'] = pd.to_datetime(df_stg_sessions['session_date'], errors='coerce')
df_stg_sessions = df_stg_sessions.drop_duplicates(subset='session_id')
print(f"STG_SESSIONS: {len(df_stg_sessions)} строк после очистки")
# ДЕТАЛЬНОЕ ЛОГИРОВАНИЕ
print(f"   📅 session_date преобразован в: {df_stg_sessions['session_date'].dtype}")
print(f"   🗑️  Удалено дубликатов: {len(loaded_tables["sessions"]) - len(df_stg_sessions)}")
print(f"   ⚠️  Пропуски UTM: {df_stg_sessions['utm_source'].isnull().sum()} source, {df_stg_sessions['utm_medium'].isnull().sum()} medium")

# STG_SUBSCRIPTIONS: статус и даты
df_stg_subs = loaded_tables["subscriptions"].copy()
df_stg_subs['start_date'] = pd.to_datetime(df_stg_subs['start_date'], errors='coerce')

# Безопасная обработка end_date (если колонка существует)
if 'end_date' in df_stg_subs.columns:
    df_stg_subs['end_date'] = pd.to_datetime(df_stg_subs['end_date'], errors='coerce')

df_stg_subs['status'] = df_stg_subs['status'].fillna('unknown').str.upper().str.strip()
print(f"STG_SUBSCRIPTIONS: {len(df_stg_subs)} строк после очистки")
# ДЕТАЛЬНОЕ ЛОГИРОВАНИЕ
print(f"   📅 start_date преобразован в: {df_stg_subs['start_date'].dtype}")
if 'end_date' in df_stg_subs.columns:
    print(f"   📅 end_date преобразован в: {df_stg_subs['end_date'].dtype}")
print(f"   🏷️  Статусы нормализованы: {df_stg_subs['status'].unique()}")
print(f"   ⚠️  Пропуски статусов: {df_stg_subs['status'].isnull().sum()}")

# STG_INVOICES: даты и суммы
df_stg_invoices = loaded_tables["invoices"].copy()
df_stg_invoices['period_start'] = pd.to_datetime(df_stg_invoices['period_start'], errors='coerce')
df_stg_invoices['period_end'] = pd.to_datetime(df_stg_invoices['period_end'], errors='coerce')
df_stg_invoices['invoice_date'] = pd.to_datetime(df_stg_invoices['invoice_date'], errors='coerce')
df_stg_invoices['amount'] = pd.to_numeric(df_stg_invoices['amount'], errors='coerce')
print(f"STG_INVOICES: {len(df_stg_invoices)} строк после очистки")
# ДЕТАЛЬНОЕ ЛОГИРОВАНИЕ
print(f"   💰 amount преобразован в: {df_stg_invoices['amount'].dtype}")
print(f"   📅 Все даты преобразованы в datetime")
print(f"   ⚠️  Отрицательные amount: {(df_stg_invoices['amount'] < 0).sum()}")
print(f"   ⚠️  Пропуски amount: {df_stg_invoices['amount'].isnull().sum()}")

# STG_EVENTS: даты и события (исправлено - используем event_name вместо event_type)
df_stg_events = loaded_tables["events"].copy()
df_stg_events['event_date'] = pd.to_datetime(df_stg_events['event_date'], errors='coerce')

# Обрабатываем event_name (а не event_type)
if 'event_name' in df_stg_events.columns:
    df_stg_events['event_name'] = df_stg_events['event_name'].fillna('unknown').str.upper().str.strip()
else:
    print("⚠️  Колонка 'event_name' отсутствует в events")

print(f"STG_EVENTS: {len(df_stg_events)} строк после очистки")
# ДЕТАЛЬНОЕ ЛОГИРОВАНИЕ
print(f"   📅 event_date преобразован в: {df_stg_events['event_date'].dtype}")
print(f"   👤 user_id преобразован в: {df_stg_events['user_id'].dtype}")
print(f"   ⚠️  Пропуски user_id: {df_stg_events['user_id'].isnull().sum()}")

# ----------------------
# 4. Записываем staging-таблицы обратно в БД
# ----------------------
staging_tables = {
    "stg_users": df_stg_users,
    "stg_sessions": df_stg_sessions,
    "stg_subscriptions": df_stg_subs,
    "stg_invoices": df_stg_invoices,
    "stg_events": df_stg_events
}

# Выводим информацию о колонках для отладки
print("\nСтруктура таблиц:")
for table_name, df in staging_tables.items():
    print(f"{table_name}: {list(df.columns)}")

for table_name, df in staging_tables.items():
    try:
        # Сохраняем с обработкой ошибок
        df.to_sql(
            table_name, 
            engine, 
            schema=DB_SCHEMA, 
            if_exists='replace', 
            index=False
        )
        print(f"✓ Таблица {DB_SCHEMA}.{table_name} сохранена ({len(df)} строк)")
        
    except Exception as e:
        print(f"✗ Ошибка сохранения {table_name}: {e}")

print("✓ Все staging-таблицы обработаны и сохранены")